In [ ]:
# Data Cleaning & Preparation
## Using Airbnb Data

<img src='airbnb_photo.JPG'>
<i>Image by <a href="https://pixabay.com/users/instagramfotografin-5746148/?utm_source=link-attribution&amp;utm_medium=referral&amp;utm_campaign=image&amp;utm_content=3399753">InstagramFOTOGRAFIN</a> from <a href="https://pixabay.com/?utm_source=link-attribution&amp;utm_medium=referral&amp;utm_campaign=image&amp;utm_content=3399753">Pixabay</a></i>

It is often said that an analyst spends 80% or more of their time on data preparation tasks such as cleaning, transforming and rearranging data. In this module, we will discuss ways to handle missing data, duplicate data, string manipulation and other types of transformations.

When we first explore a topic, we will use a simple example to explain the process. Then you will practice by implementing a similar transformation using a real world dataset in a more complex way.

For this module, we will be working with data from [Airbnb](https://www.airbnb.com/), one of the top travel websites where hosts can list their properties or rooms for vacation rental. The ultimate goal of working with this data is to see if we can predict a property's price based on its specific attributes. We will specifically be using data from Vienna, Austria.

The Airbnb city data comes from [Inside Airbnb](http://insideairbnb.com/get-the-data.html), an investigatory website that focuses on highlighting illegal renting through Airbnb and how Airbnb affects property values in an area. The data is sourced directly from the Airbnb website and is updated monthly. 

The data is ‘messy’ with long text fields, large amounts of missing data and many features that might not have any correlation with prices. An example of the ‘messiness’ of the data can be found in the ‘amenities’ feature. Each property has a list of amenities that it provides, some standard from the Airbnb website and some that are entered directly by the property owners. This data will need to be pulled apart and cleaned, with new features added based on these amenities.

**Note 1:** In order to better show examples of data transformations, some of the data has been adjusted for teaching purposes. If you want to do any further data analysis, please download the actual data from the link above.

**Note 2:** We are only using Pandas and Numpy for these data transformations. In future classes, you will learn about additional, sometimes easier, options such as using Scikit-learn for some of these transformations. But even these options use Pandas and Numpy as their base.

## Table of Contents
1. [Import Data](#import)
2. [Initial Data Exploration](#exploration)
3. [Missing Data](#missing)
[<ul>Filtering Missing Data</ul>](#filter)
[<ul>Dropping Missing Data</ul>](#drop)
[<ul>Filling Missing Data</ul>](#filling)
4. [Data Transformation](#transformation)
[<ul>Removing Duplicates</ul>](#remove_dupes)
[<ul>Transforming Data Using a Function or Mapping</ul>](#map)
[<ul>Replacing Values</ul>](#replace)
[<ul>Binning</ul>](#bin)
[<ul>Detecting & Filtering Outliers</ul>](#outlier)
[<ul>Dummy Variables</ul>](#dummy)
5. [String Manipulation](#string)
[<ul>String Object Methods</ul>](#string_object)
[<ul>Regular Expressions</ul>](#regex)
6. [Extra Practice](#import)
7. [Conclusion](#conclusion)

## Import Data<a name="import"></a>

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# set display columns to max so that we can view all columns
pd.set_option('display.max_columns', None)

# import the Vienna dataset
vienna_raw = pd.read_csv('listings_vienna.csv', index_col='id')

# saving raw data as new dataframe for exploration
vienna = vienna_raw.copy()

## Initial Data Exploration<a name="exploration"></a>

# view first few rows
vienna.head()

# view number of rows/columns
vienna.shape

vienna.info()

There are three different datatypes represented in the data. Notice that the `price` column, which will be our label (ie what we are trying to predict), is represented as a string instead of a number.  We will need to change this before running any type of machine learning algorithm.  Also, there are several features that have missing values. We'll take care of these as we go along.

We will analyze more of the features later, but let's first look at a few of the features in further detail: `neighborhoods in Vienna`, `property types`, `room types`, and `amenities`.

# view value counts for neighborhood_cleansed
# k =vienna.groupby(['neighbourhood_cleansed']).size()
# k
vienna['neighbourhood_cleansed'].value_counts()

# view value counts for property_type
vienna['property_type'].value_counts()

There are a lot of different values for this feature. We could combine some of these (example: cabin and chalet should be similar to a house) to make our list more manageable. But first, let's also look at `room_type`.

# view value counts for room_type
vienna['room_type'].value_counts()

This seems to be similar data as the `property_type`. I'll leave both in for now and you would want to check for multi-linear correlation on your own if you were actually working on this as a project.

# view value counts for amenities
vienna['amenities']

The `amenities` feature is a list of available amenities for each property. This will be difficult to work with but some of these listed amenities might have an influence on the price. We will work on separating these list items out later.

After a manual review of each feature, there are several features that we can immediately remove from the dataset. First, we can drop features that are not related to price.

drop_features = ['listing_url','scrape_id','host_url','host_name','host_location',
                'host_neighbourhood','neighbourhood_group_cleansed','calendar_updated','license',
                'host_thumbnail_url','host_picture_url','host_verifications','host_has_profile_pic',
                'has_availability','host_total_listings_count','neighbourhood',
                'minimum_minimum_nights','maximum_minimum_nights','minimum_maximum_nights','maximum_maximum_nights',
                'minimum_nights_avg_ntm','maximum_nights_avg_ntm','availability_30','availability_60',
                'availability_365','calendar_last_scraped','number_of_reviews_ltm','number_of_reviews_l30d',
                'calculated_host_listings_count_entire_homes','calculated_host_listings_count_private_rooms',
                'calculated_host_listings_count_shared_rooms','last_scraped','neighborhood_overview',
                 'host_about','picture_url','host_identity_verified', 'minimum_nights', 'maximum_nights', 
                 'availability_90', 'instant_bookable','bathrooms','host_total_listings_count']

vienna = vienna.drop(drop_features, axis=1)
vienna.head()

vienna.shape

**Important:** In a real world project, we would stop here and split the data into what is called a training set and a test set. The training set is used to explore and train the data, and the test set is used at the very end to determine how new data generalizes with your model. This will be covered in much more detail in the machine learning classes.

## Missing Data<a name="missing"></a>
Missing data is very common in real world datasets and Pandas is a powerful tool in working with this missing data.

### Filtering Missing Data<a name="filter"></a>

Let's now look at some of the missing data in the dataset. First, we need to understand what data is missing (we should attempt to either find the missing data or impute it based on other data) and what data just doesn't exist (where that fact that it is missing might provide additional information -- example: no reviews for a property).

# creates True/False mask
vienna['beds'].isnull()

vienna['beds'].isnull().sum() # missing from data

This could be an example of purely missing data. Every property probably should have at least one bed listed, even if it is a non-standard bed type.

vienna['first_review'].isnull().sum() # doesn't exist

This is an example of data that probably just doesn't exist. These properties might not have any reviews from guests. And the fact that they don't have any reviews might be important to the properties price.

### Dropping missing values<a name="drop"></a>
One way to handle missing data is to drop the values completely. This could be appropriate based on your data but remember that you might lose some other information by dropping the data.

#setup simple DataFrame
data = pd.DataFrame([[2.,4.,6.],[1.,np.nan,np.nan],
                     [np.nan,np.nan,np.nan],[np.nan,10.,12.]])
data

# using drop_na to drop rows with missing values
data.dropna()

# dropping rows where there are missing values only for column '1'
data.dropna(subset=[1])

# using drop_na to drop columns with missing values
data.dropna(axis=1)

# using drop_na to drop rows that have all missing values
data.dropna(how='all')

# using drop_na to drop rows that don't have at least 2 values
data.dropna(thresh=2)

#### Student Practice
Try to perform the following tasks on the `vienna` dataset. Then check your answers as I walk through the solutions. Unless the exercise asks you to create a new DataFrame, you can assume that you are to alter the original `vienna` data.

# run the following code
vienna.shape

**Exercise:** Create a new DataFrame called `vienna_dropped_rows` to drop all rows from the original `vienna` dataset with missing values. 

### ENTER CODE HERE ###
vienna_dropped_rows = pd.DataFrame(vienna)
vienna_dropped_rows.dropna()

**Exercise:** Create a new DataFrame called `vienna_dropped_rows_subset` where rows with missing values in the `bedrooms` column are dropped.

### ENTER CODE HERE ###
vienna_dropped_rows_subset = pd.DataFrame(vienna)
# vienna_dropped_rows_subset.loc[:,'bedrooms'].dropna()
vienna_dropped_rows_subset.dropna(subset=['bedrooms'])

**Exercise:** Create a new DataFrame called `vienna_dropped_cols` where columns with missing values are dropped.

### ENTER CODE HERE ###
veinna_dropped_cols = pd.DataFrame(vienna)
veinna_dropped_cols.dropna(axis=1)

**Exercise:** Drop rows from the `vienna` dataset that have only missing values. 

### ENTER CODE HERE ###
vienna.dropna(how='all')

**Exercise:** Drop rows from the `vienna` dataset that don't have at least 10 values.

### ENTER CODE HERE ###
vienna.dropna(thresh=10)

### Filling Missing Data<a name="filling"></a>

Instead of simply discarding data and potentially losing information from other data that goes along with it, sometimes it's better to fill the missing data. This can be done in a number of different ways.

# setup example DataFrame
data = pd.DataFrame(np.random.randn(7, 3))
data.iloc[:4, 1] = np.nan
data.iloc[:2, 2] = np.nan
data

# using fillna with a constant 
data.fillna(0)

# using fillna with a dictionary for potential different values
data.fillna({1: 1.5, 2:-1.5})

# same interpolation methods available for reindexing can be used with fillna
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = np.nan
df.iloc[4:, 2] = np.nan
df

# using 'ffill'
df.fillna(method='ffill')

# using 'ffill' with limit
df.fillna(method='ffill', limit=2)

# using the mean of each column
df.fillna(df.mean())

#### Student Practice
Try to perform the following tasks on the `vienna` dataset. Then check your answers as I walk through the solutions. Unless the exercise asks you to create a new DataFrame, you can assume that you are to alter the original `vienna` data.

# look at missing data for bathrooms
vienna['bathrooms_text'].isnull().sum()

**Exercise:** Fill missing values from `bathrooms_text` with the number `1`.

### ENTER CODE HERE ###
vienna['bathrooms_text'].fillna(1, inplace=True)
vienna['bathrooms_text'].value_counts()

**Exercise:** Fill missing values as follows:
- bedrooms: 1
- host_listings_count: 1
- host_is_superhost: 'f'

### ENTER CODE HERE ###
vienna[vienna['bedrooms'].isna() | vienna['host_listings_count'].isna() | vienna['host_is_superhost'].isna()][['bedrooms','host_listings_count','host_is_superhost']]

vienna.fillna({'bedrooms':1, 'host_listings_count':1, 'host_is_superhost':'f'}, inplace=True)


**Exercise:** Fill the missing values in the `beds` column with the median.

### ENTER CODE HERE ###
# vienna['beds'].fillna(value=vienna.median)
vienna['beds'].fillna(vienna['beds'].median(), inplace=True)
vienna['beds'].isnull().sum()

**Note:** There are more efficient ways to handle missing values by using pipelines with Scikit-learn, which you will study more in the machine learning classes. These pipelines will allow you to clean future data more easily so that you won't have to do it manually each time.

## Data Transformation<a name="transformation"></a>
### Removing Duplicates<a name="remove_dupes"></a>
You may also find yourself spending a lot of time identifying and handling duplicate data. Sometimes it will be easy to identify the duplicates (all or most of the values are the same), while other times duplicates will be much harder to identify.

# create basic DataFrame
data = pd.DataFrame({'Student': ['Stephanie','Nadia','Lukas','Sally','Nadia','Nadia'],
                    'Grade': [100,95,100,80,100,100]})
data

# returns boolean Series: True represents whether row has been observed before
data.duplicated()

# returns only non-duplicated rows
data.drop_duplicates()

# keep the last row instead of the first
data.drop_duplicates(keep='last')

# default considers all values
# can specify subset 
data.drop_duplicates(['Student'])

#### Student Practice
Try to perform the following tasks on the `vienna` dataset. Then check your answers as I walk through the solutions. Unless the exercise asks you to create a new DataFrame, you can assume that you are to alter the original `vienna` data.

**Exercise:** How many exact duplicates are in the `vienna` data? Drop any exact duplicates from the data.

### ENTER CODE HERE ###
vienna.duplicated()
vienna.drop_duplicates(inplace=True)
vienna.duplicated().sum()

**Exercise:** How many properties have the exact same `host_id`, `name`, and `description`and are in the same neighborhood as a previous property?

### ENTER CODE HERE ###
# vienna['description'].duplicated().sum() + vienna['name'].duplicated().sum()+ vienna['description'].duplicated().sum()
vienna.duplicated(['host_id', 'name','description', 'neighbourhood_cleansed']).sum()

**Exercise**: These properties may be duplicates. Let's say after researching further that we have decided to remove these duplicated rows. Remove these rows from the `vienna` data.

### ENTER CODE HERE ###
vienna.drop_duplicates(['host_id', 'name','description', 'neighbourhood_cleansed'],inplace=True)


### Transforming Data Using a Function or Mapping<a name="map"></a>
Performing transformations based on values in an array.


# create a sample DataFrame
data = pd.DataFrame({
    'city': ['atlanta','Baltimore','boston','Buffalo','charlotte'],
    'state': ['GA','Maryland','Massachusetts','New York','NC']
})

data

First, notice that some of these cities are capitlized and some are not. We need to keep that in mind when we go to map the values.

Suppose you wanted to add a column indicating the NFL mascot for the respective city. We can map the city to the mascot name as follows:

# create a dictionary mapping
city_to_mascot = {
    'atlanta':'Falcons',
    'baltimore':'Ravens',
    'boston':'Patriots',
    'buffalo':'Bills',
    'charlotte':'Panthers'
}

# use `str.lower()` to convert values before mapping
lowercased_city = data['city'].str.lower()
lowercased_city

# `map()` accepts a function or dictionary-like object
data['mascot'] = lowercased_city.map(city_to_mascot)
data

# also can pass a function
data['city'].map(lambda x: city_to_mascot[x.lower()])

Sometimes you might only want to map some values and leave others as they appear in the data. You can combine `map()` and `fillna()` for this purpose.

How could we use a mapping to abbreviate the states that don't yet have abbreviations?

state_abbr = {
    'Maryland':'MD',
    'Massachusetts':'MA',
    'New York':'NY'
}

# make a copy of data to use with the following example
data2 = data.copy()

# this won't work
data2['state']= data2['state'].map(state_abbr)
data2

data['state'] = data['state'].map(state_abbr).fillna(data['state'])
data

#### Student Practice
Try to perform the following tasks on the `vienna` dataset. Then check your answers as I walk through the solutions. Unless the exercise asks you to create a new DataFrame, you can assume that you are to alter the original `vienna` data.

**Exercise:** Change the column name from the British spelling of `neighbourhood_cleansed` to the American spelling of `neighborhood_cleansed`.

### ENTER CODE HERE ###
vienna.rename(columns={'neighbourhood_cleansed':'neighborhood_cleansed'}, inplace=True)
vienna

**Exercise:** Check the value counts of this column. Notice that the various foreign language characters did not get encoded correctly.

### ENTER CODE HERE ###
vienna['neighborhood_cleansed'].value_counts()

**Exercise:** Use a mapping to update the following neighborhood names. Check your value counts again to ensure that the names were updated. 
- Landstrasse
- Rudolfsheim-Funfhaus
- Wahring
- Dobling

### ENTER CODE HERE ###
updateNames = {'Landstra§e':'Landstrasse', 'Rudolfsheim-Fnfhaus':'Rudolfsheim-Funfhaus','Whring':'Wahring','Dbling':'Dobling' }
vienna['neighborhood_cleansed']=  vienna['neighborhood_cleansed'].map(updateNames ).fillna(vienna['neighborhood_cleansed'])
vienna['neighborhood_cleansed'].value_counts()

**Exercise:** Using Seaborn, plot a scatterplot using the `longitude` and `latitude` of the `vienna` DataFrame using the `neighborhood_cleansed` column as the hue.

### ENTER CODE HERE ###
plt.figure(figsize=(12,8))
g = sns.scatterplot(x= vienna['longitude'], y=vienna['latitude'], hue=vienna['neighborhood_cleansed'])
plt.show()
# for i in vienna.columns:
#     print(i)

### Replacing Values<a name="replace"></a>
As we saw previously, `map` can be used to modify a subset of values but `replace` can sometimes be simpler and more flexible. However, using `map` is sometimes more efficient for larger datasets.

data
data['mascot']= ['Falcons', 'Ravens', 'Patriots', 'Bills', 'Panthers']
data

# using replace to replace a single value
data.replace('Panthers', np.nan, inplace=True)
data

# using replace to replace multiple values
data.replace(['Falcons', 'Ravens'], np.nan, inplace=True)
data

# using replace to replace different values using lists
data.replace(['Patriots','Bills'],[np.nan, 'Wings'])

# using replace to replace different values using a dictionary
data.replace({'Patriots':np.nan, 'Bills':'Wings'}, inplace=True)
data

#### Student Practice
Try to perform the following tasks on the `vienna` dataset. Then check your answers as I walk through the solutions. Unless the exercise asks you to create a new DataFrame, you can assume that you are to alter the original `vienna` data.

**Exercise:** Check the value counts for the property types.

### ENTER CODE HERE ###
vienna['property_type'].value_counts()

**Exercise:** We want to combine some of these categories so that we eventually only have four categories: 'Apartment', 'House', 'Room', and 'Other'. For example, 'Entire house' and 'Entire cottage' can probably just be listed as 'House' to simplify the data.

First, combine the property types that you think could be simply represented by 'House' and 'Apartment' (Note: answers will vary based on how you break down the categories)

### ENTER CODE HERE ###Tiny house
house = ['Castle','Entire place' 'Entire hostel',
         'Entire place',                     
'Entire bungalow',                  
'Dome house',                       
'Entire hostel',    
         
         'Entire chalet', 'Casa particular', 'Lighthouse','Tiny house', 'Entire cottage','Entire villa', 'Entire guesthouse',
'Entire house']
apartment = ['Entire cabin','Entire townhouse','Entire serviced apartment','Entire guest suite', 'Camper/RV', 'Room in bed and breakfast', 'Entire loft', 'Entire condominium', 'Entire apartment']

vienna['property_type'].replace(house, 'house', inplace=True)
vienna['property_type'].replace(apartment, 'apartment', inplace=True)
vienna['property_type'].value_counts()


**Exercise:** Perform a similar task of combining property types that might be simply considered as a 'Room'. 

*Bonus:* You could perform this with `replace()` but it would take a long dictionary of values. Can you perform this using the Pandas [str.contains()](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.contains.html) method in one line of code?

### ENTER CODE HERE ###

# room = ['Private room in apartment', 'Private room in house', 'Room in boutique hotel',
#         'Private room in condominium', 'Shared room in apartment',
#         'Private room in bed and breakfast', 'Room in hotel',
#         'Private room in loft', 'Room in aparthotel','Private room in townhouse',
#         'Private room in hostel',
#         'Private room in guesthouse',
#         'Shared room in hostel',
#         'Room in bed and breakfast',
#         'Private room in serviced apartment',
#         'Private room in guest suite', 'Camper/RV', 'Shared room in condominium',
#          'Shared room in loft', 'Private room in villa', 'Private room', 'Private room in earth house',
#         'Private room in castle', 'Shared room in house',
#         'Shared room in serviced apartment', 'Shared room in bed and breakfast',
#         'Private room in farm stay', 'Private room in nature lodge',
#         'Shared room in tiny house', 'Shared room in hotel', 'Private room in camper/rv',
#         'Private room in cave', 'Private room in bus',
#         'Private room in treehouse', 'Shared room in tent']  
# vienna['property_type'].replace(room, 'room', inplace=True)
# vienna['property_type'].value_counts()



**Exercise:** Change all other values not listed as 'House', 'Apartment', or 'Room' to 'Other'. 

*Bonus:* Again, you could perform this with `replace()`, but can you think of a way to do it in one line of code? Hint: You want to select everything where the property type is not 'House', 'Apartment', or 'Room' and code those as 'Other'.   

### ENTER CODE HERE ###
vienna.loc[~vienna.property_type.isin(['room', 'house','apartment']), 'property_type']= 'Others'

vienna['property_type'].value_counts()

**Exercise:** Check the property type value counts one last time to verify that you now only have four categories.

### ENTER CODE HERE ###

### Binning<a name="bin"></a>

Continuous data is often separated into 'bins' for analysis.

# setup list of birth years
year = [1955, 1964, 1975, 1985, 1960, 2001, 1978]

# setup bins 
bins = [1945, 1964, 1980, 1996, 2012]

# use the `cut()` function to divide years into their appropriate bins
cats = pd.cut(year, bins)

# returns a special `Categorical` object
cats

Uses mathematical notation for intervals.
- `(` is *open* (not included in the category)
- `]` is *closed* (inclusive)
- can change which side is closed by passing `right=False`

# can pass your own bin names
cats = pd.cut(year, bins, labels=['Baby Boomer', 'Generation X', 
                                  'Generation Y (Millennials)', 'Generation Z'])

# label for the year data
cats.codes

# categories for the year data
cats.categories

# value counts for the bins
pd.value_counts(cats)

# create random array of data
data = np.random.rand(100)
data

# equal length bins based on min and max values
pd.cut(data, 4, precision=2)

# create sample Series data
ser1 = pd.Series(np.random.rand(20))
ser1[[1,3,5]] = np.nan
ser1

# bin in four equal categories
ser1 = pd.cut(ser1, 4, precision=4, labels=['small','medium','large','x-large'])
ser1

# notice syntax for Series
ser1.cat.codes

ser1.cat.categories

# add category - added to the front position
ser1 = ser1.cat.add_categories('not_measured')
ser1

# reorder categories so that new category is first
ser1 = ser1.cat.reorder_categories(['not_measured','small','medium','large','x-large'])
ser1

# make categories unordered
ser1 = ser1.cat.as_unordered()
ser1

# fillna with 'not_measured'
ser1 = ser1.fillna('not_measured')
#ser1 = ser1.fillna('missing')
ser1

# creating a categorical datatype with automatic categories
ser2 = pd.Series(['a','b','a','c','b','a']).astype('category')
ser2

#### Student Practice
Try to perform the following tasks on the `vienna` dataset. Then check your answers as I walk through the solutions. Unless the exercise asks you to create a new DataFrame, you can assume that you are to alter the original `vienna` data.

**Exercise:** View the `host_response_time`, `host_response_rate`, and `host_acceptance_rate` columns of the `vienna` dataset.

### ENTER CODE HERE ###

**Exercise:** For `host_response_time`, code all missing values as `unknown`

### ENTER CODE HERE ###

**Exercise:** Make the `host_response_time` a categorical datatype.

### ENTER CODE HERE ###

**Exercise:** First, run the code below to turn the `host_acceptance_rate` and `host_response_rate` from strings to floats. 

Note that the `regex=True` determines if the passed-in pattern is a regular expression:

- If True, assumes the passed-in pattern is a regular expression.

- If False, treats the pattern as a literal string

# change string values to floats
vienna.loc[:,'host_acceptance_rate'] = vienna.loc[:,'host_acceptance_rate'].str.replace('\%', '', regex=True).astype(float)
vienna.loc[:,'host_response_rate'] = vienna.loc[:,'host_response_rate'].str.replace('\%', '', regex=True).astype(float)

**Exercise:** Create the following bins for the `host_acceptance_rate` and `host_response_rate` columns
- 0 - 49
- 50 - 79
- 80 - 89
- 90 - 99
- 100

### ENTER CODE HERE ###

**Exercise:** 
1. Add a new `unknown` category for both `host_response_rate` and `host_acceptance_rate`. 
2. Re-order the categories so that `unknown` is first.
3. Fill all missing values in these two columns with the `unknown` category.

### ENTER CODE HERE ###

### Detecting & Filtering Outliers<a name="outlier"></a>

# create DataFrame of normally distributed data
data = pd.DataFrame(np.random.randn(1000, 4))
data

data.describe()

# select values in column 1 that have values exceeding 3 in absolute value
column1 = data[1]
column1[np.abs(column1) > 3]

# select all rows having a value exceeding 3 or -3

### old way to code ###
# data[(np.abs(data) > 3).any(1)]

### new way due to Pandas update ###
data[(np.abs(data) > 3).any(axis=1)]

# now set the values based on this criteria
data[np.abs(data) > 3] = np.sign(data) * 3

# look at results

### old way to code ###
# data[(np.abs(data) == 3).any(1)]

### new way due to Pandas update ###
data[(np.abs(data) == 3).any(axis=1)]

# notice min and max
data.describe() 

# np.sign() produces 1 and -1 values based on positive/negative
np.sign(data).head()

Now, let's look at the `price` column in the `vienna` dataset.

# view price
vienna['price']

# remove dollar signs and commas; change to float
vienna['price'] = vienna['price'].str.replace('\$', '', regex=True).str.replace(',','').astype(float)

# verify results
vienna['price']

# let's plot the price data
sns.kdeplot(data=vienna['price'], fill='fill') # updated code due to seaborn update
plt.show()

# check min price
vienna['price'].min()

# remove instances with $0 price
vienna = vienna.drop(vienna[vienna['price'] == 0].index)

vienna['price'].min()

# example to explain previous index code 
vienna[vienna['price'] == 9.0].index

# check max price
vienna['price'].max()

# 99% of data fall below the following price
top = int(vienna['price'].quantile(0.99))
top

# remove top 1% outliers
vienna = vienna.drop(vienna[vienna['price'] > top].index)

vienna['price'].max()

# check new kde plot with outliers removed
sns.kdeplot(data=vienna['price'], fill='fill') # updated code due to seaborn update
plt.show()

### Dummy Variables<a name="dummy"></a>

Convert categorical variables into dummy variables as a lot of machine learning algorithms work specifically with numerical data.

# create a simple series
ser = pd.Series(['dog','cat','dog','horse'])
ser

pd.get_dummies(ser)

pd.get_dummies(ser, prefix='animal')

# reduces correlation and dimensionality of data
pd.get_dummies(ser, drop_first=True)

ser1 = ['cat','dog',np.nan]
ser1

pd.get_dummies(ser1)

pd.get_dummies(ser1, dummy_na=True)

#### Student Practice
Try to perform the following tasks on the `vienna` dataset. Then check your answers as I walk through the solutions. Unless the exercise asks you to create a new DataFrame, you can assume that you are to alter the original `vienna` data.

**Exercise:** What are the values and counts for the following three columns: `host_is_superhost`, `room_type`, and `property_type`?

### ENTER CODE HERE ###

**Exercise:** Use `get_dummies()` with the `room_type` and `property_type`columns. Add a prefix of your choosing and drop the first dummy variable for each. *Note: I didn't show you how to work with columns within a DataFrame. See if you can look at the [pandas documentation](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html) to figure this out so that you update the `vienna` DataFrame.*

### ENTER CODE HERE ###

**Exercise:** To make sure you understand what `get_dummies()` is doing, can you take the `host_is_superhost` column and turn the values into dummy variables without using `get_dummies()`? Make sure that you only have one column called `superhost` after you are finished and the original column is deleted.

### ENTER CODE HERE ###

## String Manipulation<a name="string"></a>

### String Object Methods<a name="string_object"></a>

One of Pythons most popular strengths is its ability to work with strings and text. Some of the more simple operations can easily be done with the built in string methods.

# create simple string of text
a = 'Jimmy,Anderson,  jimmy@gmail.com'
a

# using the split() method
a.split(',') # splits text on the comma

# strip() removes whitespace from around character
b = [x.strip() for x in a.split(',')]
b

# joining strings within a list
'++'.join(b)

# count of number of commas
a.count(',')

# replace strings in text
a.replace(',','')

There are many more [string methods](https://www.w3schools.com/python/python_ref_string.asp) that you can research, and you will learn more about some of these in the 575 Python class if you haven't already taken it.

Now, let's try to clean up the 'messy' `amenities` column from the `vienna` data. 

# view the amenities for one property
vienna.iloc[0]['amenities']

# create an empty list
all_amenities = []

# iterate through rows, extending amenities to list
for row in vienna.amenities:
    all_amenities.extend(row.replace('[','').replace(']','').replace('"','').lower().strip().split(','))

all_amenities

amenities_list = pd.unique(all_amenities)
amenities_list

### Note: You are not responsible for knowing how to do this part of the code. This is to show you what can be done. ###
amenity_values = vienna['amenities']
amenity_values

# check the following for more information: https://docs.python.org/3/library/collections.html#collections.Counter
from collections import Counter

# create Series of amenities column
amenity_values = vienna['amenities']

# instantiate counter object
counter = Counter()

# update counter with each amenity and its total
for response in amenity_values:
    counter.update(response.replace('[','').replace(']','').replace('"','').lower().split(','))

# create two empty lists    
amenities_list = []
amenities_count = []

# iterate through most common amenities in counter and append to lists
for item in counter.most_common(50):
    amenities_list.append(item[0])
    amenities_count.append(item[1])
    
most_common_amenities = zip(amenities_list,amenities_count)
print(list(most_common_amenities))

def amenities_to_columns(data):
    '''
    Creates new amenity features if the list of amenities for a respective property includes
    certain key words.
    
    Returns: DataFrame
    '''
    # create list of new amenity columns
    new_columns = ['tv','longterm','washing','cooking','fridge','fireplace','free_parking','paid_parking',
                   'air_conditioning','balcony','water_access','coffee','breakfast','family','workspace']
    
    #search 'amenities' feature for keywords, create new column and dummy variable if true
    data.loc[data['amenities'].str.contains('tv|cable|amazon prime|netflix', case=False), 'tv'] = 1
    data.loc[data['amenities'].str.contains('long term', case=False), 'longterm'] = 1
    data.loc[data['amenities'].str.contains('washer|dishwasher', case=False), 'washing'] = 1
    data.loc[data['amenities'].str.contains('stove|oven|microwave', case=False), 'cooking'] = 1
    data.loc[data['amenities'].str.contains('refrigerator|freezer|fridge', case=False), 'fridge'] = 1
    data.loc[data['amenities'].str.contains('fireplace|fire pit', case=False), 'fireplace'] = 1
    data.loc[data['amenities'].str.contains('free parking|free street parking', case=False), 'free_parking'] = 1
    data.loc[data['amenities'].str.contains('paid parking|paid street parking',case=False), 'paid_parking'] = 1
    data.loc[data['amenities'].str.contains('air conditioning|central air conditioning', case=False), 'air_conditioning'] = 1
    data.loc[data['amenities'].str.contains('balcony|patio', case=False), 'balcony'] = 1
    data.loc[data['amenities'].str.contains('hot tub|waterfront|pool|lake|beachfront', case=False), 'water_access'] = 1
    data.loc[data['amenities'].str.contains('coffee|coffee machine|nespresso', case=False), 'coffee'] = 1
    data.loc[data['amenities'].str.contains('breakfast', case=False), 'breakfast'] = 1
    data.loc[data['amenities'].str.contains('high chair|crib|children|child|baby|family', case=False), 'family'] = 1
    data.loc[data['amenities'].str.contains('workspace', case=False), 'workspace'] = 1

    # replace na's with 0
    data[new_columns] = data[new_columns].fillna(0)
    
    return data
    
vienna = amenities_to_columns(vienna)

vienna = vienna.drop('amenities', axis=1)
vienna.head()

### Regular Expressions<a name="regex"></a>

While string methods are powerful and can be used in a lot of different ways, sometimes you need a way to search a more complex string pattern. This is where regular expressions (`regex`) offer a flexible way to search or match complex patterns.

Regex could be an entire module on its own, but we will cover some of the basics now.

https://www.dataquest.io/wp-content/uploads/2019/03/python-regular-expressions-cheat-sheet.pdf

# import the re module
import re

# create a simple text message
text = 'A     Lannister\tnever\nforgets'

# split strings based on whitespace characters (tabs, spaces, newlines)
re.split('\s+', text)

When you call `re.split()` it is first compiled, then the split method is called on the passed text. You can compile the regex yourself, which is highly recommended if you apply the same expression to many strings. This will also save you CPU cycles. 

# setup text of names and emails
emails = """
James james.anderson@eastern.edu
Sally sally@gmail.com
Ryan ryan22@yahoo.com
"""

# creating pattern using raw string literal
pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b' 

# compile the regex
regex = re.compile(pattern, flags=re.IGNORECASE)

# returns all matches in a string
regex.findall(emails)

# returns only the first match
position = regex.search(emails) # first email, start and end position
position

emails[position.start():position.end()]

# only matches the beginning of the string
print(regex.match(emails)) #only match if pattern occurs at the start of the string

# simple program to validate an email

#create email list
email_list = ['james.anderson@eastern.edu','sally@gmail.com','ryan22@yahoo']

# iterate through emails
for email in email_list:
    if (re.fullmatch(regex, email)):
        print("Valid Email")
 
    else:
        print("Invalid Email")
        

# returns new string with occurrences of the pattern replaced by the new string
print(regex.sub('REDACTED', emails))

# wrap parentheses around different segments
pattern2 = r'(\b[A-Za-z0-9._%+-]+)@([A-Za-z0-9.-]+)\.([A-Z|a-z]{2,}\b)'

# compile with new pattern
regex = re.compile(pattern2, flags=re.IGNORECASE)

# returns tuple of matched components
segments = regex.match('jamie.andrews@eastern.edu')
segments

segments.groups()

Let's look at a practical example using our `vienna` dataset. We need to turn the strings as listed in the `bathrooms_text` column to floats.

# view column
vienna['bathrooms_text'].value_counts()

pattern = r"[-+]?\d*\.\d+|\d+"
# [-+] --> handles plus or minus signs
# ? -->  matches the expression to its left 0 or 1 times.
# \d --> matches digits
# * --> matches the expression to its left 0 or more times.
# \. --> escapes the decimal character
# + --> y matches the expression to its left 1 or more times.
# | --> matches expression before or after 


def bathroomtext_to_num(text):
    '''
    Strips out number from bathrooms_text field 
    
    Returns: float
    '''
    pattern = r"[-+]?\d*\.\d+|\d+"  
    regex = re.compile(pattern)
    num_baths = regex.findall(str(text))
    
    if 'half' in str(text).lower():
        return 0.5
    elif num_baths == []:
        return 1
    else:
        return num_baths[0]

# apply custom function to 'bathrooms_text' column
vienna['bathrooms'] = vienna.apply(lambda row: bathroomtext_to_num(row['bathrooms_text']), axis=1).astype('float64')


# view new 'bathrooms' feature
vienna[['bathrooms','bathrooms_text']].head(10)

vienna = vienna.drop('bathrooms_text', axis=1)

vienna.head()  

## Extra Practice: Plotting and Miscellaneous<a name='extra'></a>

**Exercise:** Plot a [scatterplot](https://seaborn.pydata.org/generated/seaborn.scatterplot.html) of longitude and latitude using the 'free_parking' attribute as the color. What do you notice about properties with free parking?

### ENTER CODE HERE ###

**Exercise:** Plot a scatterplot of longitude and latitude using price as the color. Do you notice any discerable pattern in terms of price versus the area of Vienna?

### ENTER CODE HERE ###

**Exercise:** Plot a [violin plot](https://seaborn.pydata.org/generated/seaborn.violinplot.html) using the price and the neighborhood name. What neighborhoods seem to have the largest range and highest median prices?

### ENTER CODE HERE ###

**Exercise:** What areas have the most listings? Create a [horizontal bar chart](https://seaborn.pydata.org/generated/seaborn.countplot.html) to show these listings by neighborhood.

### ENTER CODE HERE ###

**Exercise:** Using a pivot table, show the top hosts (represented by `host_id`) that have the most multiple listings (`calculated_host_listings_count`).

### ENTER CODE HERE ###

**Exercise:** How many beds, bedrooms, and bathrooms do most properties have? What percentage of properties have those respective most common values?

### ENTER CODE HERE ###

**Exercise:** Similar to how we binned the `host_response_time`, `host_response_rate`, and `host_acceptance_rate`, perform a similar binning for the various review scores attributes.
    
- `review_scores_rating`: bin the review scores between '0-79', '80-94','95-99' and '100'.
- other review ratings: bin these scores between '0-8','9', and '10'
- add an unknown category and fill the na's with 'unkown' (don't forget to reorder your categories so that 'unknown' is first)

### ENTER CODE HERE ###

## Conclusion<a name="conclusion"></a>

In reality, a lot more needs to be done to this dataset before it is ready for a machine learning algorithm to be run. Also, as I mentioned above, there are sometimes easier ways to perform some of these data manipulations using Scikit-learn, which you will learn about more in the machine learning classes.

After cleaning up the data further, checking for correlation between the attributes, dropping attributes and creating some new ones using this data, the best machine learning model was able to predict **66%** of the price using this data with a RMSE score of **0.3643** (the target value was changed to use the log of the price).

Given the above results, my conclusion is that this data can be used to predict prices somewhat but more work or more data is needed to create a better model. Future work could involve:

- performing analysis on the photos and determine if they have an impact on price
- performing a sentiment analysis on the foreign language comments (from a different file on the InsideAirbnb website) to determine if the guest comments could be an indicator of price. I was able to perform a sentiment analysis on the English language comments and there was not much impact on the model.
- further feature extraction/elimination
- prices are the listed prices from the Airbnb website. A more accurate model might be obtained by using the prices that guests actually paid for a property
- personally scrape data from the Aribnb website to obtain more information than what is listed in the file